In [1]:
from random import shuffle
import glob, h5py, cv2
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
width = 128
height = 128
channels = 3
trainpath = '/mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/*.png'
sub = 'single_act'
filename = 'datasets/'+sub+'.hdf5'
print('File to be created: ' + filename)

File to be created: datasets/single_act.hdf5


In [20]:
# load single act images
train_files2 = sorted(glob.glob(trainpath))
print(len(train_files2))
train_files = []
fname_ls = []
sing_angs_ls = []
for i, file in enumerate(train_files2):
    
    fname = train_files2[i].split('/')[-1]
#     print(train_files2[i])
    angles = fname.split('_')[2][3:]
    plusses = angles.rfind('+')
    minuses = angles.rfind('-')

    if plusses == -1 and minuses == -1:
        all_ang_ids = int(angles)
        angs = all_ang_ids
    elif plusses == -1 and minuses != -1:
        all_ang_ids = sorted([minuses, len(angles)+1])
    elif plusses != -1 and minuses == -1:
        all_ang_ids = sorted([plusses, len(angles)+1])
    else:
        all_ang_ids = sorted([plusses, minuses, len(angles)+1])

    if isinstance(all_ang_ids, list):
        if all_ang_ids[0] == 0:
            angs = np.zeros((len(all_ang_ids) - 1,), dtype=int)
            for i in range(len(all_ang_ids) - 1):
                angs[i] = int(angles[all_ang_ids[i]:all_ang_ids[i+1]]) # .astype(int)
        else:
            angs = np.zeros((len(all_ang_ids),), dtype=int)
            angs[0] = int(angles[:all_ang_ids[0]])
            for i in range(len(all_ang_ids) - 1):
                angs[i+1] = int(angles[all_ang_ids[i]:all_ang_ids[i+1]]) # .astype(int)
              
    if not isinstance(angs, int): # if multi-target
        if len(angs) == 1:
            angs = int(angs[0])
        else:
            continue
    
    sing_angs_ls.append(angs) 
#     print(file)
    train_files.append(file)
print('Num. of files: '+str(len(train_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+train_files[i])

679
Num. of files: 658
1. /mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pAhmed_class10_ang0_iter1.png
2. /mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pAhmed_class10_ang0_iter2.png
3. /mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pAhmed_class10_ang0_iter3.png
4. /mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pAhmed_class15_ang0_iter1.png
5. /mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pAhmed_class15_ang0_iter2.png


In [25]:
labels = []
files = []
class_ls = []
orig_classes = [1,2,3,4,5,6,7,13,14]
cnt = 1
for i, file in enumerate(train_files):
    clear_output(wait=True)
    print('Loading training labels #'+str(cnt))
    fname = file.split('/')[-1]
    clas = int(fname.split('_')[1][5:])
    class_ls.append(clas)
    
    if clas not in orig_classes:
        continue
    if clas == 13:
        clas = 8
    elif clas == 14:
        clas = 9
    clas -= 1
    labels.append(clas)
    files.append(file)
    cnt += 1
labels2 = np.array(labels)
print('Orig classes', orig_classes)
print('New classes:', np.unique(labels))
labels = to_categorical(labels2)

print(labels.shape)

Loading training labels #476
Orig classes [1, 2, 3, 4, 5, 6, 7, 13, 14]
New classes: [0 1 2 3 4 5 6 7 8]
(476, 9)


In [26]:
Counter(labels2)

Counter({0: 69, 1: 69, 4: 72, 5: 69, 6: 69, 7: 30, 8: 30, 2: 38, 3: 30})

In [27]:
cnt = 1
images = []

for image in files:
    img = cv2.imread(image)
    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Loading image '+str(cnt)+'/'+str(len(files))+ ' '+image)
    images.append(img)  
    cnt += 1
    
images = np.swapaxes(np.array(images), 1, 2).reshape(len(files), width, height, channels)/255.
images.shape

Loading image 476/476 /mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pSpot_class4_ang0_iter9.png


(476, 128, 128, 3)

In [29]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=1)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(380, 128, 128, 3)
(380, 9)
(96, 128, 128, 3)
(96, 9)


In [30]:
f = h5py.File(filename, mode='w')
f.create_dataset("train_img", x_train.shape, np.uint8)
f.create_dataset("test_img", x_test.shape, np.uint8)  
f.create_dataset("train_labels", y_train.shape, np.uint8)
f.create_dataset("test_labels", y_test.shape, np.uint8)

f["train_img"][...] = x_train
f["test_img"][...] = x_test
f["train_labels"][...] = y_train
f["test_labels"][...] = y_test
f.close()
print(filename+' created.')

datasets/single_act.hdf5 created.
